# Predict abc notes based on 8 bars input using BERT 

In [181]:
from Preprocessing_notebook import read_abc
from Transformer_model import  get_model
from tqdm import tqdm
from pathlib import Path
import torch
import youtokentome as yttm
from argparse import ArgumentParser
import os 
ORIGIN = os.path.normpath(os.getcwd() + os.sep + os.pardir)
print(ORIGIN)
RUN5_DIR = ORIGIN +"/Music/MusicTransformer/Run_5/221394_134131.abc"

TOKENIZER_DIR = ORIGIN +"/Music/maestro-v3.0.0/new_abc_notitle.yttm"
TOKENIZER_DIR = ORIGIN +"/Music/maestro-v3.0.0/abc_newDS.yttm"

OUTPUT_DIR = ORIGIN +"/Music/maestro-v3.0.0/output"
OUTPUT_PREDICT_DIR = ORIGIN +"/Music/maestro-v3.0.0/output/predict"
model_path = "/Users/adam/Desktop/Adam/Masters_Courses/Semster_3/Music/pytorch_model_FINAL_newDS.bin"


/Users/adam/Desktop/Adam/Masters_Courses/Semster_3


In [77]:
test_paths = TEST_DIR
test_paths = sorted(test_paths)

print("Loading tokenizer...")
tokenizer = yttm.BPE(TOKENIZER_DIR) # import the trained tokenizer

model = get_model(vocab_size=tokenizer.vocab_size()) # load the BERT model 



checkpoint = torch.load(model_path, map_location="cpu")
model.load_state_dict(checkpoint)



Loading tokenizer...


<All keys matched successfully>

In [3]:
USEABLE_PARAMS = [i+":" for i in "BCDFGHIKLMmNOPQRrSsTUVWwXZ"] # These are the parameters for key

def read_abc(path):
    keys = []
    notes = []
    with open(path) as rf:
        for line in rf:
            line = line.strip()
            if line.startswith("%"): # Skip any commments
                continue

            if any([line.startswith(key) for key in USEABLE_PARAMS]):
                if(line.startswith('T')):
                    continue # skipping the title for better tokenization
                keys.append(line)
            else:
                notes.append(line)

    keys = " ".join(keys)

    notes = "".join(notes).strip()
    notes = notes.replace(" ", "")

    if notes.endswith("|"):
        notes = notes[:-1]
    # Remove unneeded character.
    notes = notes.replace(" \ ", "")
    notes = notes.replace("\\", "")
    notes = notes.replace("\ ", "")
    notes = notes.replace("x8|", "") # 8 because all of the midi file has a L:1/8 that means one muted bar
    notes = notes.replace("z8|", "") # 8 because all of the midi file has a L:1/8 that means one muted bar

    notes = notes.strip()
    notes = " ".join(notes.split(" "))

    if not keys or not notes:
        return None, None

    return keys, notes



In [60]:


def predict_notes(model, tokenizer, keys, notes):
    print(notes)
    keys_tokens = tokenizer.encode(keys)
    notes_tokens = tokenizer.encode(notes)

    if len(keys_tokens) + len(notes_tokens) > 510:
        notes_tokens = notes_tokens[len(notes_tokens) - len(keys_tokens) - 510:]

    context_tokens = [2] + keys_tokens + notes_tokens + [3]

    context_tokens = torch.tensor(context_tokens, dtype=torch.long).unsqueeze(0)


    
    bad_words_ids = []
    bad_words = ["x8 | "]
    for w in bad_words:
        bad_words_ids.append(tokenizer.encode(bad_words)[0])

    gen_tokens = model.generate(input_ids=context_tokens, 
                            max_length=80, 
                            min_length=40,
                            num_beams=70, # num of memorizing sequences
                            bos_token_id=2, 
                            eos_token_id=3,
                            no_repeat_ngram_size=2, # allows to avoid melody repeating
                            pad_token_id=0,
                            temperature = 0.5,
                            top_k = 5,
                                
                            bad_words_ids=bad_words_ids)
                                
    gen_tokens = gen_tokens[0].tolist()

    notes = tokenizer.decode(gen_tokens, ignore_ids=[0,1,2,3])[0]
    notes = notes.replace(" ", "").replace("|", "|\n")
    
    return notes

def predict(model, tokenizer, text_path, output_dir):
    text_path = TEST_DIR
    files = os.listdir(text_path)

    for i,file in enumerate(files):
        print(file)
        if(file == '.DS_Store'):
            continue
        keys, notes = read_abc(RUN5_DIR)
        print(keys)
        print(notes)
        if notes is None:
            print(keys)
            continue
            
    #keys, notes = read_abc(text_path)
    print(keys)


    # Find the index of the 8th '|' character
    index = 0
    count = 0
    for i, char in enumerate(notes):
        if char == '|':
            count += 1
            if count == 8:
                index = i
                break

# Remove the text after the 8th '|' character
    orignal_notes = notes[index:]

    notes = notes[:index+1]    
    
    new_path = text_path

    predicted_tokens = predict_notes(model, tokenizer, keys, notes)


    return [predicted_tokens, orignal_notes]
        


In [ ]:
import nltk
import Levenshtein

def calculate_bleu_score(reference, generated):

    reference_tokens = reference.split()
    generated_tokens = generated.split()

    reference_corpus = [reference_tokens]

    # Calculate the BLEU score based on 1gram 2gram and 3gram
    bleu_score_ngram = nltk.translate.bleu_score.corpus_bleu(reference_corpus, generated_tokens)
    bleu_score_2ngram = nltk.translate.bleu_score.corpus_bleu(reference_corpus, generated_tokens, weights=(0.5, 0.5))
    bleu_score_3grams = nltk.translate.bleu_score.corpus_bleu(reference_corpus, generated_tokens, weights=(0.33, 0.33, 0.33, 0))
    print(f"1gram : {bleu_score_ngram}, 2grams: {bleu_score_2ngram}, 3grams{bleu_score_3grams}")
    return bleu_score

def levenshtein(abc_1 ,abc_2):
    original = abc_1
    predicted = abc_2

    # Calculate Levenshtein distance
    distance = Levenshtein.distance(abc_1, abc_2)

    # Normalize the distance to a similarity score
    max_length = max(len(original), len(predicted))
    similarity_score = 1 - (distance / max_length)

    print(f"Similarity Score: {similarity_score:.2f}")


In [183]:
print("Starts generation")
output_dir = Path(TEST_DIR)
output_dir.mkdir(exist_ok=True)
for p in tqdm(test_paths):
    print(p)
    abc_path = predict(model, tokenizer, p, output_dir)
    print('predicted')
    print(abc_path[0])
    print('orginal')
    print(abc_path[1])
    
    print(bars_similarity(str(abc_path[0]),str(abc_path[1])))
    bleu_score = calculate_bleu_score(abc_path[0], abc_path[1])
    print("BLEU Score:", bleu_score)
    levenshtein(abc_path[0],abc_path[1])
    break

Starts generation


  0%|                                                    | 0/84 [00:00<?, ?it/s]

-
.DS_Store
7_162982.abc
/Users/adam/Desktop/Adam/Masters_Courses/Semster_3/Music/maestro-v3.0.0/test_1_path/7_162982.abc
X:1 M:4/4 L:1/8 Q:1/4=222 K:E%4sharps V:1
A,,2-[e4A4E4A,,4]E,2-|[a2e2A2E,2]A,-[a3-e3-A3-A,3-][a/2e/2-A/2-A,/2E,/2-][e/2A/2-E,/2-][A/2E,/2-]E,/2|E,,2-[e3-B3-E3-E,,3-][e/2-B/2E/2E,,/2-][e/2E,,/2]B,,2-|[g3/2-e3/2-B3/2B,,3/2-][g/2e/2B,,/2]E,-[g-e-B-E,-][g/2-e/2-B/2-E,/2F,,/2-][g3/2e3/2B3/2-F,,3/2][B/2G,,/2-]G,,3/2|A,,2-[e4A4E4A,,4]E,2-|[a3/2e3/2-A3/2E,3/2-][e/2E,/2]A,-[a3-e3-A3-A,3][aeA-E,-][A/2E,/2-]E,/2|A,,2-[e4A4E4A,,4]E,2-|[a2e2A2E,2-][A,/2-E,/2]A,/2-[a3-e3-A3-A,3-][a/2-e/2-A/2-A,/2E,/2-][aeA-E,-][A/2E,/2]|E,,2-[e3-B3-E3-E,,3-][e/2-B/2E/2E,,/2-][e/2E,,/2]B,,2-|[g3/2-e3/2-B3/2B,,3/2-][g/2e/2B,,/2]E,-[g4e4B4-E,4-][B/2E,/2]x/2|B,,3-[b3-d3-F3-B,,3-][b/2-d/2-F/2-B,,/2B,,,/2-][b-d-FB,,,-][b/2d/2B,,,/2-]|[b3/2f3/2B3/2F3/2B,,,3/2]x6x/2|[B,,8-E,,8-]|[B,,4-E,,4-][e4B,,4-E,,4-]|[g3-B3-B,,3-E,,3-][g/2B/2B,,/2-E,,/2-][B,,/2-E,,/2-][g4-B4-B,,4-E,,4-]|[g6-B6-B,,6-E,,6-][g3/2B3/2B,

  0%|                                                    | 0/84 [00:27<?, ?it/s]

predicted
a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2[E-D-][E2D2C2][E-D-][E2D2C2]][c/2A/2F/2][c/2A/2F/2][c/2A/2F/2][c/2A/2F/2F2D2B,2][F2D2B,2][A2-E2-A,2-][A2-E2-A,2-][_G/2-E/2-B,/2][B/2-__G/2-E/2-B,/2][B/2-_G,3/2-_E,3/2-][G,3/2-_E,3/2-][,/2x/2[,/2x/2[g/2-d/2-G/2-D/2-B,/2G,/2-D,/2-B,,/2G,,/2-][g/2d/2G/2-D/2g/2-d/2-G/2-D/2-B,/2G,/2-D,/2-B,,/2G,,/2-][g/2d/2G/2-D/2G/2-E/2-=B,/2-E,,/2][G/2E/2=B,/2G/2-E/2-=B,/2-E,,/2][G/2E/2=B,/2[B3/2-G3/2-[B3/2-G3/2-ED-ED-^F,,3/2][^F,,3/2][[=D/2-A,/2-_G,/2-D,,/2][D/2-A,/2-_G,/2-A,,/2][D/2A,/2_G,/2D,/2]x/2[[=D/2-A,/2-_G,/2-D,,/2][D/2-A,/2-_G,/2-A,,/2][D/2A,/2_G,/2D,/2]x/2[F,2-C,2-][A,2F,2-C,2-][B,2F,2-C,2-][F,2-C,2-][A,2F,2-C,2-][B,2F,2-C,2-][A,/2][c'/2A,/2][c'/2G/2E/2C/2G,/2C,/2][G/2E/2C/2G,/2C,/2][A,/2-F,/2-C,/2F,,/2][A,/2-F,/2-C,/2F,,/2][d2B2E2d2B2E2c/2F/2C/2c/2F/2C/2E/2B,/2G,/2E,/2-E,,/2-][E,/2E,,/2-][C/2E,/2E,,/2]x/2[E/2B,/2G,/2E,/2-E,,/2-][E,/2E,,/2-][C/2E,/2E,,/2]x/2[^A,/2^G,

In [67]:
print("Starts generation")
output_dir = Path(TEST_DIR)
output_dir.mkdir(exist_ok=True)
counter = 0
for p in tqdm(test_paths):
    print(p)
    results = predict(model, tokenizer, p, output_dir)
    print(results[0])
    print("========")
    print(results[1])
    counter =counter+1
    if counter ==2:
        break

Starts generation


  0%|                                                    | 0/84 [00:00<?, ?it/s]

-
.DS_Store
7_162982.abc
/Users/adam/Desktop/Adam/Masters_Courses/Semster_3/Music/maestro-v3.0.0/test_1_path/7_162982.abc
X:1 M:4/4 L:1/8 Q:1/4=120 K:F%1flats V:1
G,2x2x/2D2-D/2G-|G/2F/2-[G/2-F/2]G/2A/2G/2-[B/2-G/2]B/2x/2B,B,3/2B|x/2B>GB<GE3/2D2|ED3/2G-[G/2E/2]x/2G/2-[G/2E/2-][E/2D/2]x/2=B,/2-[D/2-=B,/2][D/2C/2-]|C/2=B,<G,G,,3/2G,3/2xD/2=B,-|=B,/2C2D<ED3-D/2|x/2E3D2<D,2D/2-|D_B3/2B,-[B/2B,/2]xGx/2G,3/2|G3/2E-[E/2D/2-]D3/2E/2x/2D3/2G-|[G/2E/2]x/2G/2-[G/2E/2-]E/2(3D=B,DC/2-[C/2=B,/2-]=B,/2G,-[G,/2G,,/2-]G,,/2-|G,,/2G,2=B,<ED/2x/2G/2-[G/2E/2-][E/2D/2]x/2=B,/2-|=B,C4Ex/2=B,3/2|(3D2C2=B,2(3G,2D2F2|Gx/2Ax/2(3_B2G2E2D-|DE(3D2G2E2D3/2C/2-|C(3=B,2G,2G,,2G,>DG
X:1 M:4/4 L:1/8 Q:1/4=120 K:F%1flats V:1
G,2x2x/2D2-D/2G-|G/2F/2-[G/2-F/2]G/2A/2G/2-[B/2-G/2]B/2x/2B,B,3/2B|x/2B>GB<GE3/2D2|ED3/2G-[G/2E/2]x/2G/2-[G/2E/2-][E/2D/2]x/2=B,/2-[D/2-=B,/2][D/2C/2-]|C/2=B,<G,G,,3/2G,3/2xD/2=B,-|=B,/2C2D<ED3-D/2|x/2E3D2<D,2D/2-|D_B3/2B,-[B/2B,/2]xGx/2G,3/2|


  1%|▌                                           | 1/84 [00:23<32:12, 23.28s/it]

F,/2B,,/2-]B,,/2F,/2B,,/2-]B,,/2_D,_D,C,-]C,C,-]C,E/2-A,,/2]E/2-[E/2-A,,/2]E/2-[A,E,A,,]xx/2x/2[CG,C,]x/2x/2[A,E,A,,]xx/2x/2[CG,C,]x/2x/2[E/2-C/2-A,/2E,/2-A,,/2-][E/2-C/2-E,/2-A,,/2-][E/2-C/2-A,/2E,/2-A,,/2-][E/2-C/2-E,/2-A,,/2-][(3ddd(3dddd2B2][d2B2][F-A,-F,,][F-A,-F,,][C,,/2-]C,C,,/2-]C,_E-C-G,-_E-C-G,-B,E,B,E,^F^D^F^D=E/2x/2[=E/2x/2[[GEC[GEC[D/2B,/2][F/2D/2]B/2[F/2D/2][B/2F/2][d/2B/2[D/2B,/2][F/2D/2]B/2[F/2D/2][B/2F/2][d/2B/2e/2c/2]x/2[e/2c/2e/2c/2]x/2[e/2c/2g/2f/2e/2d/2g/2f/2e/2d/2C,/2]x3C,/2]x3[^G-EB,]^G/2[E/2B,/2^G,/2][^G-EB,]^G/2[E/2B,/2^G,/2][A-EC]A/2[[^G-EB,]^G/2[E/2B,/2^G,/2][^G-EB,]^G/2[E/2B,/2^G,/2][A-EC]A/2[][f/2B/2][f/2B/2^F/2D/2^F/2D/2d/2B/2G/2-D/2-B,/2-G,/2-E,/2-][d/2B/2G/2-D/2-B,/2-G,/2-E,/2-][G/2-E/2-B,/2][G/2-E/2-B,/2][^D,/2]x[^D,/2]x[G2C2A,2G2C2A,2D3-B,3-][D3-B,3-][[d3A3F3]x/2[[d3A3F3]x/2[C/2A,/2F,/2-][C/2A,/2F,/2-][B,/2-G,/2-G,,/2-][B,/2-G,/2-G,,/2-][F/2C/2A,/2][F/2C/2A,/2][F/2C/2A,/2][F/2C/2A,/2][F/2C/2A,/2][F/2C/2A,/2][F/2C/2A,/2][F/2C/2A,/2][F/2C/2A,/2][F/2C/2A,

  1%|▌                                         | 1/84 [00:47<1:05:03, 47.03s/it]

a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2G/2-E/2-C/2-C,/2][G/2E/2C/2G/2-E/2-C/2-C,/2][G/2E/2C/2]F,,]F,,G-E-C][G-E-C][f/2d/2=A/2f/2d/2=A/2D3/2-A,3/2-D3/2-A,3/2-=d/2F/2-A,/2-][F/2-A,/2-][a/2g/2f/2=d/2F/2-A,/2-][F/2-A,/2-][a/2g/2f/2B/2=F/2B/2=F/2[AEC[AECG,/2E,/2C,/2][G,/2E,/2C,/2][G,/2E,/2C,/2][G,/2E,/2C,/2][B,,/2-][E/2B,,/2-][B,,/2-][E/2B,,/2-][D,/2]A/2-[A/2-D,/2]A/2-[A/2-,/2x/2[,/2x/2[F/2D/2B,/2F,/2][F/2D/2B,/2F/2D/2B,/2F,/2][F/2D/2B,/2g/2G/2g/2G/2F-F,-][F-F,-][GECG,-C,-][GECG,-C,-][F,/2-]F,/2-F,/2-]F,/2-E2-C2-A,2-][E/2C/2A,/2][E2-C2-A,2-][E/2C/2A,/2][=C/2A,/2=C/2A,/2C2-A,2-F,2-][C2-A,2-F,2-][c/2G/2E/2A,/2-]A,/2c/2G/2E/2A,/2-]A,/2[D2B,2G,2E,2A,,2]x[D3/2B,3/2G,3/2A,,3/2]x/2[D/2B,/2G,/2A,,/2]x/2[D2B,2G,2[D2B,2G,2E,2A,,2]x[D3/2B,3/2G,3/2A,,3/2]x/2[D/2B,/2G,/2A,,/2]x/2[D2B,2G,2C2-C/2C2-C/2D/2_A,/2D/2_A,/2AE^CAE^C[E2C2A,2[E2C2A,2B-B-G3-F3-D3-][G3-F3-D3-][g3-g3-[AFC-F,-][[AFC-F,-][[E3/2[E3/2d/2B/2E/2]x[d/2B/2E/2]x3

In [187]:
import nltk

def calculate_bleu_score(reference, generated):
    # Tokenize the reference and generated sequences
    reference_tokens = reference.split()
    generated_tokens = generated.split()

    # Create a list of reference token lists (BLEU expects a list of lists)
    reference_corpus = [reference_tokens]

    # Calculate the BLEU score
    bleu_score_ngram = nltk.translate.bleu_score.corpus_bleu(reference_corpus, generated_tokens)
    bleu_score_2ngram = nltk.translate.bleu_score.corpus_bleu(reference_corpus, generated_tokens, weights=(0.5, 0.5))
    bleu_score_3grams = nltk.translate.bleu_score.corpus_bleu(reference_corpus, generated_tokens, weights=(0.33, 0.33, 0.33, 0))
    print(f"1gram : {bleu_score_ngram}, 2grams: {bleu_score_2ngram}, 3grams{bleu_score_3grams}")
    return bleu_score

# Example usage
#215688_124450
# abc_1 ="|[_A2-G2-D2-A,2][AGD-]D/2x3/2[F/2D/2_B,/2]x3/2[F/2D/2B,/2]x/2|x2[G3/2E3/2B,3/2]x3/2[F/2D/2B,/2]x3/2[F/2D/2B,/2]x/2|x2[G3/2E3/2B,3/2]x3/2[F/2D/2B,/2-]B,/2x[F/2D/2B,/2]x/2|x2[G3/2E3/2B,3/2]x3/2[F/2D/2-B,/2]D/2x[F/2D/2B,/2]x/2|x2[GEB,]x2[C/2A,/2F,/2]x3/2[C/2A,/2F,/2]x/2|x2[D3/2B,3/2F,3/2]x3/2[C/2A,/2F,/2]x3/2[C/2A,/2F,/2]x/2|x2[D3/2B,3/2F,3/2]x3/2[C/2A,/2F,/2]x3/2[C/2A,/2F,/2]x/2|x2[D3/2B,3/2F,3/2]x3/2[C/2A,/2F,/2]x3/2[C/2A,/2F,/2]x/2"
# abc_2_bert = "F,/2B,,/2-]B,,/2F,/2B,,/2-]B,,/2D/2-G,/2-D/2-G,/2-E2][dD][cC][E2][dD][cC][G,/2-]G,/2-G,/2-]G,/2-c/2-A/2-F/2-C/2-A,/2c/2-A/2-F/2-C/2-A,/2B,,/2E,,/2][B,,E,,][B,,E,,][B,,/2E,,/2]E,/2B,,/2E,,/2][B,,E,,][B,,E,,][B,,/2E,,/2]E,/2^d/2-=^d/2-=A,4D,4][A,4D,4][=B,-F,-][=B,-F,-][f/2f/2f/2d/2f/2f/2f/2f/2f/2d/2f/2f/2E,C,E,C,B,/2G,/2E,/2C,/2-]C,/2[B,/2G,/2E,/2C,/2-]C,/2[B,/2G,/2E,/2C,/2-]C,/2[B,/2-G,/2E,/2C,/2]B,/2G,/2E,/2C,/2-]C,/2[B,/2G,/2E,/2C,/2-]C,/2[B,/2G,/2E,/2C,/2-]C,/2[B,/2-G,/2E,/2C,/2]E,/2A,,/2]x/2[E,/2A,,/2]x/2[F,["

# abc_2 ="x2[DB,F,]x2B,-[DB,]F-|[f2-d2-B2-F2][f2-d2-B2-][f/2d/2-B/2-][d/2B/2]A,-[=B,A,]E-|[e2-=B2-A2-E2-][e/2-=B/2-A/2-E/2][e=B-A-][=B/2A/2]xE,-[G,-E,][=A,G,]|[c4-=A4-G4-E4-C4-][c=A-G-E-C][=A/2G/2E/2D,/2-]D,/2-[G,D,-][_A,/2-D,/2]A,/2-|[A2-G2-D2-A,2][A3/2G3/2D3/2]x3/2_B,-[D-B,-][F-D-B,-]|[f3-d3-B3-F3-D3-B,3][f/2d/2B/2F/2D/2]x4x/2|A,,-[_D-A,,-][E-_D-A,,-][A2-E2-_D2-A,,2-][A/2-E/2-_D/2-A,,/2][A3/2-E3/2-_D3/2][A-E-]|A/2-E/2=D,/2-][A/2-D,/2-][A-F,-D,-][A-A,-F,D,][A-A,-][A-D-A,-][A-F-DA,][A/2F/2]x/2|A,,-[F,-A,,-][A,-F,-A,,-][d-AFA,-F,-A,,-][d/2A,/2-F,/2-A,,/2-][A,/2-F,/2A,,/2][A"
#234675_21548
# abc_1 = "|[E4-C4-A,4-][ECA,-][E2A,2-][EA,]|[E8-B,8-A,8-]|[E4-B,4A,4-][E-A,-][ECA,-][EA,-][GA,]|[B3E3-A,3-][A3E3-A,3-][B2E2-A,2-]|[c4-=B4-_B4-E4-A,4-][c=B_BE-A,-][BE-A,-][AE-A,-][fEA,]|[e8-E8-A,8-]|[e8E8A,8]|x[eE][e4-E4-][eE][B-B,-]"
# abc_bert_2 = "F,/2B,,/2-]B,,/2F,/2B,,/2-]B,,/2_D,_D,B,/2G,/2E,,/2-]E,,/2-[B,/2G,/2E,,/2-]E,,/2-[e/2-c/2-G/2-C/2][e/2c/2G/2e/2-c/2-G/2-C/2][e/2c/2G/2A-DA-D=CA,=CA,^G/2^G,/2-][^G/2^G,/2-][_E2C2_E2C2E3/2B,3/2G,3/2-E,3/2-B,,3/2-][E3/2B,3/2G,3/2-E,3/2-B,,3/2-][[c6-[c6-E^CG,-D,-G,,-][E^CG,-D,-G,,-][F,6F,6F,2C,2-F,,2][F,2C,2-F,,2][6G,6-6G,6-[B2-B,2-^G,2-][B-B,-^G,-][B-B,^G,][B-DB,]B-[B-D-B,-][BDB,-[B2-B,2-^G,2-][B-B,-^G,-][B-B,^G,][B-DB,]B-[B-D-B,-][BDB,-[ge[ge][e/2E/2][e/2E/2CG,-B,,][CG,-B,,][C/2A,/2F,/2-]F,/2[C/2A,/2F,/2-]F,/2[C/2-A,/2-E,/2-][C/2A,/2E,/2][C/2-A,/2-E,/2-][C/2A,/2E,/2][G/2D/2-B,/2G/2D/2-B,/2[B,3F,3-B,,3][[B,3F,3-B,,3][e-cAe-cAE,-][eE,-][eE/2-C/2-E,,/2-][E/2-C/2-E,,/2-][^A,,/2-][F/2^A,,/2-]"
# abc_gpt2_2 = "E8-C2-C-C/2-C/2-C/2F,/2-][C/2-F,/2-]|C/2-F-C/2-F,/2-F,/2-][F-CF,][F2-C2-F,2-][F/2-C/2-F,/2][F/2-C/2G,/2-][F/2-G,/2-][F/2A,/2-G,/2F,/2-][A,/2-F,/2][A,/2-G,/2-]|A,-[C-A,-][F-C-A,-][c/2-F/2-C/2-A,/2][c/2-F/2C/2][c/2G,/2-]G,3/2-[D-G,-"

#106429_180318
# abc_1 = "|[A2-F2-D2-D,,2-][A/2-F/2-D/2-D,,/2][A/2-F/2-D/2-][A2-F2-D2-D,,2][A3-F3-D3-]|[A3/2-F3/2-E3/2D3/2-B,,,3/2-][A/2-F/2-D/2-B,,,/2][A-F-D-][A3/2-F3/2-D3/2-B,,,3/2][A2-F2-D2-][A/2-F/2-D/2-][A-F-DB,,,-]|[A/2-F/2E/2-C/2-B,,,/2A,,,/2-][A3/2-E3/2-C3/2-A,,,3/2-][A/2-F/2-E/2-C/2-A,,,/2][A/2-F/2E/2-C/2-][A2-E2-C2-A,,,2][A-E-C-A,-][A/2-E/2-D/2-C/2-A,/2][A/2-E/2-D/2C/2-][A-E-C-]|[A/2-F/2-E/2C/2-F,,/2-][A3/2-F3/2-C3/2-F,,3/2][A-F-C-G,,-][A/2-F/2-C/2-A,,/2-G,,/2][A/2-F/2-C/2-A,,/2-][A-GF-EC-A,,][A-F-C-C,][A3/2F3/2C3/2F,3/2]x/2|[A2-F2-D2-D,,2-][A/2-F/2-D/2-D,,/2][A/2-F/2-D/2-]["
# abc_2 = "F,/2B,,/2-]B,,/2F,/2B,,/2-]B,,/2d/2-A/2-F/2-D/2-d/2-A/2-F/2-D/2-A2F2C2A2F2C2A3/2F3/2-D3/2-A,3/2-A,,3/2-][A3/2F3/2-D3/2-A,3/2-A,,3/2-][G/2-B,/2G,/2][G/2-B,/2G,/2][E3/2C3/2][E3/2C3/2][[A2F2D2A,2]F,[A2F2D2A,2]F,G,2E,2][G,2E,2][B/2G,/2-]G,/2-B/2G,/2-]G,/2-B/2-G/2-D/2-][B/2G/2D/2B/2-G/2-D/2-][B/2G/2D/2AF,-F,,-][cAF,-F,,-][cB,,/2-][A/2B,,/2-][B,,/2-][A/2B,,/2-][f2c2A2F2C2f2c2A2F2C2G/2-B,/2-G,/2G/2-B,/2-G,/2A,-F,-C,-F,,-][A,-F,-C,-F,,-][=G,,/2-]G,,/2x3=G,,/2-]G,,/2x3,/2B,,,,/2B,,,,/2B,,,,/2B,,,^AGEC-G,-C,-][=A^FDC-G,-C,-][^AGCG,C,][^AGEC-G,-C,-][=A^FDC-G,-C,-][^AGCG,C,]"
# abc_2 = "F,,,-[B/2F/2C/2F,,,/2]x/2[B/2F/2C/2F,,,/2-]F,,,/2-[B/2F/2C/2F,,,/2]x/2[B-F-C][B/2F/2]x/2[AFC-]C/2x/2|[F-DB,F,,,-][F/2F,,,/2]x/2[FDB,F,,,-]F,,,/2x/2[F-D-B,][F/2-D/2C,,/2][F/2D,,/2][G/2-E/2-C/2-F,,/2-][G/2-E/2-C/2F,,/2D,,/2][G/2E/2C,,/2]A,,,/2|F,,,-[B/2F/2C/2F,,,/2]x/2[B/2F/2C/2F,,,/2-]F,,,/2-[B/2F/2C/2F,,,/2]x/2[B-F-C][B/2F/2]x/2[A-FC-][A/2C/2]x/2|[F-D-B,D,,-][F/2D/2D,,/2-]D,,/2-[F-DB,D,,-][F/2D,,/2-][D/2D,,/2][F-D-B,C,,-][A/2-F/2D/2C,,/2-][A/2C,,/2-][G-ECC,,-][G/2F/2-C,,/2-][F/2C,,/2]|[B-F-DB,,,-][B/2F/2B,,,/2-]B,,,/2-[G/2-F/2D/2B,,,/2]G/2[B/2-F/2-D/2-B,,,/2][B/2-F/2-"
#191238_36222

#221394_134131
abc_1 = "|E,,2-[e3-B3-E3-E,,3-][e/2-B/2E/2E,,/2-][e/2E,,/2]B,,2-|[g3/2-e3/2-B3/2B,,3/2-][g/2e/2B,,/2]E,-[g4e4B4-E,4-][B/2E,/2]x/2|B,,3-[b3-d3-F3-B,,3-][b/2-d/2-F/2-B,,/2B,,,/2-][b-d-FB,,,-][b/2d/2B,,,/2-]|[b3/2f3/2B3/2F3/2B,,,3/2]x6x/2|[B,,8-E,,8-]|[B,,4-E,,4-][e4B,,4-E,,4-]|[g3-B3-B,,3-E,,3-][g/2B/2B,,/2-E,,/2-][B,,/2-E,,/2-][g4-B4-B,,4-E,,4-]|[g6-B6-B,,6-E,,6-][g3/2B3/2B,,3/2-E,,3/2-][B,,/2E,,/2]"
abc_2 = "a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2a/2e/2]x/2[d'/2[E-D-][E2D2C2][E-D-][E2D2C2]][c/2A/2F/2][c/2A/2F/2][c/2A/2F/2][c/2A/2F/2F2D2B,2][F2D2B,2][A2-E2-A,2-][A2-E2-A,2-][_G/2-E/2-B,/2][B/2-__G/2-E/2-B,/2][B/2-_G,3/2-_E,3/2-][G,3/2-_E,3/2-][,/2x/2[,/2x/2[g/2-d/2-G/2-D/2-B,/2G,/2-D,/2-B,,/2G,,/2-][g/2d/2G/2-D/2g/2-d/2-G/2-D/2-B,/2G,/2-D,/2-B,,/2G,,/2-][g/2d/2G/2-D/2G/2-E/2-=B,/2-E,,/2][G/2E/2=B,/2G/2-E/2-=B,/2-E,,/2][G/2E/2=B,/2[B3/2-G3/2-[B3/2-G3/2-ED-ED-^F,,3/2][^F,,3/2][[=D/2-A,/2-_G,/2-D,,/2][D/2-A,/2-_G,/2-A,,/2][D/2A,/2_G,/2D,/2]x/2[[=D/2-A,/2-_G,/2-D,,/2][D/2-A,/2-_G,/2-A,,/2][D/2A,/2_G,/2D,/2]x/2[F,2-C,2-][A,2F,2-C,2-][B,2F,2-C,2-][F,2-C,2-][A,2F,2-C,2-][B,2F,2-C,2-][A,/2][c'/2A,/2][c'/2G/2E/2C/2G,/2C,/2][G/2E/2C/2G,/2C,/2][A,/2-F,/2-C,/2F,,/2][A,/2-F,/2-C,/2F,,/2][d2B2E2d2B2E2c/2F/2C/2c/2F/2C/2E/2B,/2G,/2E,/2-E,,/2-][E,/2E,,/2-][C/2E,/2E,,/2]x/2[E/2B,/2G,/2E,/2-E,,/2-][E,/2E,,/2-][C/2E,/2E,,/2]x/2[^A,/2^G,/2F,,/2-][C/2F,,/2-][^D/2^C/2F,,/2-][F/2F,,/2-][^G/2^F/2=F,,/2-][^A/2F,,/2-][=c/2F,,/2-][^c/2F,,/2-][^d/2^A,/2^G,/2F,,/2-][C/2F,,/2-][^D/2^C/2F,,/2-][F/2F,,/2-][^G/2^F/2=F,,/2-][^A/2F,,/2-][=c/2F,,/2-][^c/2F,,/2-][^d/2_D,/2E,/2_D,/2E,/2A,2-A,,2-][A,2-A,,2-][[F/2D/2B,/2]x3/2[F/2D/2B,/2F,/2]xF,/2[F/2D/2B,/2]x[F/2D/2B,/2]x3/2[F/2D/2B,/2F,/2]xF,/2[F/2D/2B,/2]xE3C3B,3G,3E3C3B,3G,3[D/2-B,/2-][F3/2-D3/2B,3/2][B/2-F/2[D/2-B,/2-][F3/2-D3/2B,3/2][B/2-F/2A2^A2^[c6-=A[c6-=AB/2-]B/2x/2B/2-]B/2x/2D/2-B,/2-][D-B,-G,-D,-][G/2-D/2B,/2-G,/2D,/2][G/2-B,/2-][GD/2-B,/2-][D-B,-G,-D,-][G/2-D/2B,/2-G,/2D,/2][G/2-B,/2-][GB,/2-E,/2-][E/2-B,/2-E,/2-][G/2-E/2-B,/2E,/2-][G/2-E/2E,/2-][B,/2-E,/2-][E/2-B,/2-E,/2-][G/2-E/2-B,/2E,/2-][G/2-E/2E,/2-][e-c-A][e-c-A][][A/2A,/2][][A/2A,/2][cG-E-cG-E-[E-B,-E,-][[E-B,-E,-][]x/2[F/2C/2A,/2]x/2[]x/2[F/2C/2A,/2]x/2[D,-[F,/2D,/2][D,-[F,/2D,/2][A,/2-F,/2^D,/2]A,/2-F,/2^D,/2]"
abc_2 = "A,,2-[e4A4E4A,,4]E,2-|[a2e2A2E,2]A,-[a3-e3-A3-A,3-][a/2e/2-A/2-A,/2E,/2-][e/2A/2-E,/2-][A/2E,/2-]E,/2|E,,2-[e3-B3-E3-E,,3-][e/2-B/2E/2E,,/2-][e/2E,,/2]B,,2-|[g3/2-e3/2-B3/2B,,3/2-][g/2e/2B,,/2]E,-[g-e-B-E,-][g/2-e/2-B/2-E,/2F,,/2-][g3/2e3/2B3/2-F,,3/2][B/2G,,/2-]G,,3/2|A,,2-[e4A4E4A,,4]E,2-|[a3/2e3/2-A3/2E,3/2-][e/2E,/2]A,-[a3-e3-A3-A,3][aeA-E,-][A/2E,/2-]E,/2|A,,2-[e4A4E4A,,4]E,2-|[a2e2A2E,2-][A,/2-E,/2]A,/2-[a3-e3-A3-A,3-][a/2-e/2-A/2-A,/2E,/2-][aeA-E,-][A/2E,/2]|E,,2-[e3/2-B3/2-E3/2-E,,3/2][e/2B/2-E/2][B/2B,,/2-]B,,3/2-[g-e-B-B,,][geB-E]|B/2[E,/2A,,/2-]A,,-[e4-A4-E4-A,,4-A,,4-][e,,4-][e4-]|e/2-A,,4-]e/2-A,,/2-A,,4-]|e/2-]e/2-A-A-]e/2-A-A/2-A-A-A-E,/2-A-A-A-e/2-"
# Calculate BLEU score
bleu_score = calculate_bleu_score(abc_1, abc_2)
print("BLEU Score:", bleu_score)


1gram : 0.38582220354401836, 2grams: 0.4881704186809272, 3grams0.4371174935203348
BLEU Score: 0.5568104387716536


In [188]:
import Levenshtein

def levenshtein(abc_1 ,abc_2):
# Original and predicted ABC notation phrases
    original = abc_1
    predicted = abc_2

    # Calculate Levenshtein distance
    distance = Levenshtein.distance(abc_1, abc_2)

    # Normalize the distance to a similarity score
    max_length = max(len(original), len(predicted))
    similarity_score = 1 - (distance / max_length)

    print(f"Similarity Score: {similarity_score:.2f}")


In [189]:
levenshtein(abc_1,abc_2)

Similarity Score: 0.39
